In [1]:
from pprint import pprint

In [2]:
# Get the Latin Library corpus

# ^^^ Add step for installing Latin Library as a corpus

from cltk.corpus.latin import latinlibrary
files = latinlibrary.fileids()

print(files[:15])

['12tables.txt', '1644.txt', 'abbofloracensis.txt', 'abelard/dialogus.txt', 'abelard/epistola.txt', 'abelard/historia.txt', 'addison/barometri.txt', 'addison/burnett.txt', 'addison/hannes.txt', 'addison/machinae.txt', 'addison/pax.txt', 'addison/praelium.txt', 'addison/preface.txt', 'addison/resurr.txt', 'addison/sphaer.txt']


In [3]:
print([file for file in files if 'prop' in file])

['alcuin/propos.txt', 'prop2.txt', 'prop3.txt', 'prop4.txt', 'propertius1.txt', 'tertullian/tertullian.propheta.txt']


In [4]:
print([file for file in files if 'tib' in file])

['erasmus/antibarb.txt', 'suetonius/suet.tib.txt', 'suetonius/suet.tibullus.txt', 'tibullus1.txt', 'tibullus2.txt', 'tibullus3.txt']


In [15]:
# Get the Propertius texts

propertius_file = 'propertius1.txt'
propertius_raw = latinlibrary.raw(propertius_file)

print(propertius_raw[:500])

SEXTI PROPERTI ELEGIARVM LIBER PRIMVS

1 2 3 4 5 6 7 8a 8b 9 10 11 12 13 14 15 16 17 18 19 20 21 22

I

Cynthia prima suis miserum me cepit ocellis,
    contactum nullis ante cupidinibus.
tum mihi constantis deiecit lumina fastus
    et caput impositis pressit Amor pedibus,
donec me docuit castas odisse puellas                 5
    improbus, et nullo vivere consilio.
ei mihi, iam toto furor hic non deficit anno,
    cum tamen adversos cogor habere deos.
Milanion nullos fugiendo, Tulle, labores



In [43]:
# Get the Tibullus texts

tibullus_file = 'tibullus1.txt'
tibullus_raw = latinlibrary.raw(tibullus_file)

print(tibullus_raw[:500])

Tibullus Book I
		 

		 
		 
	 
	
 

 A. TIBVLLI ELEGIAE LIBER PRIMVS 

 

 
 I II III IV V VI VII VIII IX X 

 

 
 I 

 
 Divitias alius fulvo sibi congerat auro 
 &nbsp Et teneat culti iugera multa soli, 
Quem labor adsiduus vicino terreat hoste, 
 &nbsp Martia cui somnos classica pulsa fugent: 
Me mea paupertas vita traducat inerti,    5 
 &nbsp Dum meus adsiduo luceat igne focus. 
Ipse seram teneras maturo tempore vites 
 &nbsp Rusticus et facili grandia poma manu; 
Nec spes destituat, sed 


In [44]:
print(tibullus_raw[-500:])

ellam 
 &nbsp Verberat: e caelo deripit ille deos.    60 
Sit satis e membris tenuem rescindere vestem, 
 &nbsp Sit satis ornatus dissoluisse comae, 
Sit lacrimas movisse satis: quater ille beatus, 
 &nbsp Quo tenera irato flere puella potest. 
Sed manibus qui saevus erit, scutumque sudemque    65 
 &nbsp Is gerat et miti sit procul a Venere. 
At nobis, Pax alma, veni spicamque teneto, 
 &nbsp Perfluat et pomis candidus ante sinus.
 

 

 
 
 
	 Tibullus 
	 The Latin Library 
	 The Classics Page


In [45]:
import re
import html

from cltk.stem.latin.j_v import JVReplacer
replacer = JVReplacer()

In [142]:
# Preprocess texts

def preprocess(text):

    remove_list = [r'\bThe Latin Library\b',
                   r'\bThe Classics Page\b',
                   r'\bPropertius\b',
                   r'\bSEXTI PROPERTI ELEGIARVM LIBER PRIMVS\b',
                   r'8a', r'8b', r'VIIIA', r'VIIIB',
                   r'\bTibullus Book I\b',
                   r'\bTibullus',
                   r'\bA. TIBVLLI ELEGIAE LIBER PRIMVS\b',
                   r'I II III IV V VI VII VIII IX X',
                   r'25a', r'25b'
                  ]
    
    for pattern in remove_list:
        text = re.sub(pattern, '', text)
    
    # REMOVE LETTER HEADINGS?
    
    text = html.unescape(text) # Handle html entities
    text = re.sub(r'&nbsp;?', ' ',text) #&nbsp; stripped incorrectly in corpus?
    text = re.sub(r'\x00',' ',text) #Another space problem?
    text = re.sub(r' \xa0 ', '    ', text)
    
    # Remove roman numeral headings; must be before lower & replacer
    text = re.sub(r'\b(M{1,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|C?D|D?C{1,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,4})|M{0,4}(CM|CD|D?C{0,3})(XC|X?L|L?X{1,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|I?V|V?I{1,3}))\b',' ',text)
    
    text = text.lower()
    text = replacer.replace(text) #Normalize u/v & i/j
    
    punctuation ="\"#$%&\'()+,-/:;<=>@[\]^_`{|}~.?!«»—"
    translator = str.maketrans({key: " " for key in punctuation})
    text = text.translate(translator)
    
    translator = str.maketrans({key: " " for key in '0123456789'})
    text = text.translate(translator)
    
    # Handle spacing
    text = re.sub('[ ]+',' ', text) # Remove double spaces
    text = re.sub('\t',' ', text) # Remove tabs
    text = re.sub('^\s+','', text)
    text = re.sub(' \n', '\n', text)
    text = re.sub('\n\n', '~', text)
    text = re.sub('~+', '\n\n', text)
    #text = re.sub('\s+\n+\s','\n', text) # Remove double lines and trim spaces around new lines
    
    return text.strip()

In [143]:
tibullus_edit = preprocess(tibullus_raw)


In [144]:
tibullus = tibullus_edit.split('\n\n')

In [145]:
print(len(tibullus))

10


In [146]:
propertius_edit = preprocess(propertius_raw)

print(propertius_edit)

cynthia prima suis miserum me cepit ocellis
 contactum nullis ante cupidinibus
tum mihi constantis deiecit lumina fastus
 et caput impositis pressit amor pedibus
donec me docuit castas odisse puellas
 improbus et nullo uiuere consilio
ei mihi iam toto furor hic non deficit anno
 cum tamen aduersos cogor habere deos
milanion nullos fugiendo tulle labores
 saeuitiam durae contudit iasidos
nam modo partheniis amens errabat in antris
 rursus in hirsutas ibat et ille feras
ille etiam hylaei percussus uulnere rami
 saucius arcadiis rupibus ingemuit
ergo uelocem potuit domuisse puellam
 tantum in amore fides et benefacta ualent
in me tardus amor non ullas cogitat artes
 nec meminit notas ut prius ire uias
at uos deductae quibus est pellacia lunae
 et labor in magicis sacra piare focis
en agedum dominae mentem conuertite nostrae
 et facite illa meo palleat ore magis
tunc ego crediderim manes et sidera uobis
 posse cytinaeis ducere carminibus
aut uos qui sero lapsum reuocatis amici
 quaerite no

In [147]:
propertius = propertius_edit.split('\n\n')

In [148]:
for poem in propertius:
    print(poem[:poem.find('\n')])

cynthia prima suis miserum me cepit ocellis
quid iuuat ornato procedere uita capillo
qualis thesea iacuit cedente carina
quid mihi tam multas laudando basse puellas
quid tibi uis insane meae sentire furores
non ego nunc hadriae uereor mare noscere tecum
dum tibi cadmeae dicuntur pontice thebae
tune igitur demens nec te mea cura moratur
hic erit hic iurata manet rumpantur iniqui
dicebam tibi uenturos irrisor amores
o iucunda quies primo cum testis amori
ecquid te mediis cessantem cynthia baiis
quid mihi desidiae non cessas fingere crimen
tu quod saepe soles nostro laetabere casu
tu licet abiectus tiberina molliter unda
saepe ego multa tuae leuitatis dura timebam
quae fueram magnis olim patefacta triumphis
et merito quoniam potui fugisse puellam
haec certe deserta loca et taciturna querenti
non ego nunc tristis uereor mea cynthia manes
hoc pro continuo te galle monemus amore
tu qui consortem properas euadere casum
qualis et unde genus qui sint mihi tulle penates


In [150]:
tibullus_titles = ['Tib. 1.1', 'Tib. 1.2', 'Tib. 1.3', 'Tib. 1.4', 'Tib. 1.5', 
                   'Tib. 1.6', 'Tib. 1.7', 'Tib. 1.8', 'Tib. 1.9', 'Tib. 1.10']

propertius_titles = ['Prop. 1.1', 'Prop. 1.2', 'Prop. 1.3', 'Prop. 1.4', 'Prop. 1.5', 
                     'Prop. 1.6', 'Prop. 1.7', 'Prop. 1.8a', 'Prop. 1.8b', 'Prop. 1.9', 
                     'Prop. 1.10', 'Prop. 1.11', 'Prop. 1.12', 'Prop. 1.13', 'Prop. 1.14', 
                     'Prop. 1.15', 'Prop. 1.16', 'Prop. 1.17', 'Prop. 1.18', 'Prop. 1.19', 
                     'Prop. 1.20', 'Prop. 1.21', 'Prop. 1.22']


23


In [151]:
print(propertius[0])

cynthia prima suis miserum me cepit ocellis
 contactum nullis ante cupidinibus
tum mihi constantis deiecit lumina fastus
 et caput impositis pressit amor pedibus
donec me docuit castas odisse puellas
 improbus et nullo uiuere consilio
ei mihi iam toto furor hic non deficit anno
 cum tamen aduersos cogor habere deos
milanion nullos fugiendo tulle labores
 saeuitiam durae contudit iasidos
nam modo partheniis amens errabat in antris
 rursus in hirsutas ibat et ille feras
ille etiam hylaei percussus uulnere rami
 saucius arcadiis rupibus ingemuit
ergo uelocem potuit domuisse puellam
 tantum in amore fides et benefacta ualent
in me tardus amor non ullas cogitat artes
 nec meminit notas ut prius ire uias
at uos deductae quibus est pellacia lunae
 et labor in magicis sacra piare focis
en agedum dominae mentem conuertite nostrae
 et facite illa meo palleat ore magis
tunc ego crediderim manes et sidera uobis
 posse cytinaeis ducere carminibus
aut uos qui sero lapsum reuocatis amici
 quaerite no

In [152]:
from cltk.tokenize.word import WordTokenizer
word_tokenizer = WordTokenizer('latin')

In [153]:
tokens = word_tokenizer.tokenize(propertius[0])
print(tokens)

['cynthia', 'prima', 'suis', 'miserum', 'me', 'cepit', 'ocellis', 'contactum', 'nullis', 'ante', 'cupidinibus', 'tum', 'mihi', 'constantis', 'deiecit', 'lumina', 'fastus', 'et', 'caput', 'impositis', 'pressit', 'amor', 'pedibus', 'donec', 'me', 'docuit', 'castas', 'odisse', 'puellas', 'improbus', 'et', 'nullo', 'uiuere', 'consilio', 'ei', 'mihi', 'iam', 'toto', 'furor', 'hic', 'non', 'deficit', 'anno', 'cum', 'tamen', 'aduersos', 'cogor', 'habere', 'deos', 'milanio', '-ne', 'nullos', 'fugiendo', 'tulle', 'labores', 'saeuitiam', 'durae', 'contudit', 'iasidos', 'nam', 'modo', 'partheniis', 'amens', 'errabat', 'in', 'antris', 'rursus', 'in', 'hirsutas', 'ibat', 'et', 'ille', 'feras', 'ille', 'etiam', 'hylaei', 'percussus', 'uulnere', 'rami', 'saucius', 'arcadiis', 'rupibus', 'ingemuit', 'ergo', 'uelocem', 'potuit', 'domuisse', 'puellam', 'tantum', 'in', 'amore', 'fides', 'et', 'benefacta', 'ualent', 'in', 'me', 'tardus', 'amor', 'non', 'ullas', 'cogitat', 'artes', 'nec', 'meminit', 'notas

In [158]:
propertius_tokens = [word_tokenizer.tokenize(poem) for poem in propertius]
tibullus_tokens = [word_tokenizer.tokenize(poem) for poem in tibullus]

In [161]:
import os
from cltk.utils.file_operations import open_pickle

# Set up training sentences

rel_path = os.path.join('~/cltk_data/latin/model/latin_models_cltk/lemmata/backoff')
path = os.path.expanduser(rel_path)

# Check for presence of latin_pos_lemmatized_sents
file = 'latin_pos_lemmatized_sents.pickle'      

latin_pos_lemmatized_sents_path = os.path.join(path, file)
if os.path.isfile(latin_pos_lemmatized_sents_path):
    latin_pos_lemmatized_sents = open_pickle(latin_pos_lemmatized_sents_path)
else:
    latin_pos_lemmatized_sents = []
    print('The file %s is not available in cltk_data' % file)  

In [162]:
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer
lemmatizer = BackoffLatinLemmatizer(latin_pos_lemmatized_sents)

In [163]:
lemmas = lemmatizer.lemmatize(propertius_tokens[0])

In [164]:
print(lemmas)

[('cynthia', 'cynthia'), ('prima', 'primus'), ('suis', 'suus'), ('miserum', 'miser'), ('me', 'ego'), ('cepit', 'capio'), ('ocellis', 'ocellus'), ('contactum', 'contingo'), ('nullis', 'nullus'), ('ante', 'ante'), ('cupidinibus', 'cupido'), ('tum', 'tum'), ('mihi', 'ego'), ('constantis', 'consto'), ('deiecit', 'deicio'), ('lumina', 'lumen'), ('fastus', 'fastus'), ('et', 'et'), ('caput', 'caput'), ('impositis', 'impono'), ('pressit', 'premo'), ('amor', 'amor'), ('pedibus', 'pes'), ('donec', 'donec'), ('me', 'ego'), ('docuit', 'doceo'), ('castas', 'castus1'), ('odisse', 'odi'), ('puellas', 'puella'), ('improbus', 'improbus'), ('et', 'et'), ('nullo', 'nullus'), ('uiuere', 'uiuo'), ('consilio', 'consilis'), ('ei', 'is'), ('mihi', 'ego'), ('iam', 'iam'), ('toto', 'totus'), ('furor', 'furor'), ('hic', 'hic'), ('non', 'non'), ('deficit', 'deficio'), ('anno', 'annus'), ('cum', 'cum2'), ('tamen', 'tamen'), ('aduersos', 'aduerto'), ('cogor', 'cogo'), ('habere', 'habeo'), ('deos', 'deus'), ('milani

In [165]:
propertius_lemmas = [lemmatizer.lemmatize(tokens) for tokens in propertius_tokens]
tibullus_lemmas = [lemmatizer.lemmatize(tokens) for tokens in tibullus_tokens]

KeyboardInterrupt: 

In [166]:
# ^^^ pickle the lemmatized texts to speed up workshop

In [ ]:
from collections import Counter

# Build counter of top token counts
prop_lemmas = [item[1] for item in propertius_lemmas]

propertius_lemmas_counter = Counter(prop_lemmas)
propertius_lemmas_mc = propertius_lemmas_counter.most_common(100)

running = 0

print('Top 25 lemmas in Propertius 1:\n')
print("{number:>5}  {lemma:<12}{count:<12}{percent:<12}{running:<12}".format(number="", token="lemma", count="COUNT", percent="Type-Tok %", running = "RUNNING %"))
for i, pair in enumerate(propertius_lemmas_mc[:10]):
    running += pair[1]
    print("{number:>5}. {lemma:<12}{count:<12}{percent:<12}{running:<12}".format(number=i+1, lemma=pair[0], count=pair[1], percent=str(round(pair[1] / len(propertius_lemmas)*100, 2))+"%", running = str(round(running / len(propertius_lemmas)*100, 2))+"%"))